In [13]:
source("get_final_dataset.R")

In [ ]:
old <- Sys.time()
cat("Get raw impact...\n")
impact <- read.table(paste0(data_path, "/all_IMPACT_mutations_180508.txt"),
                     sep = "\t", stringsAsFactors = FALSE, header = TRUE)
new <- Sys.time() - old
print(new)

In [ ]:
old <- Sys.time()
cat("\nGet impact_annotated (impact annotated with click_annotvcf)...\n")
impact_annotated <- get_impact_annotated()
new <- Sys.time() - old
print(new)

In [ ]:
old <- Sys.time()
cat("\nJoin impact and impact_annotated...\n")
impact <- add_click_annotvcf_annotations(impact, impact_annotated)
new <- Sys.time() - old
print(new)

In [ ]:
old <- Sys.time()
cat("\nFilter impact...\n")
impact <- filter_impact(impact)
new <- Sys.time() - old
print(new)

In [ ]:
old <- Sys.time()
cat("\nProcess raw features...\n")
impact <- process_raw_features(impact)
new <- Sys.time() - old
print(new)

In [14]:
cat("\n", nrow(impact))

Get raw impact...


Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,cDNA_change,HGVSp_Short,t_depth,t_vaf,t_alt_count,n_depth,n_vaf,n_alt_count,t_ref_plus_count,t_ref_neg_count,t_alt_plus_count,t_alt_neg_count,occurence_in_normals,confidence_class,variant_status,sample_coverage,minor_contamination
SPEN,0,MSK-IMPACT,hg19,1,16265908,16265908,+,nonsynonymous_SNV,SNP,A,A,T,novel,P-0000004-T01-IM3,NA,A,A,c.10981A>T,p.I3661F,473,0.15433,73,1071,0.00000,0,166,234,37,36,0,AUTO_OK,NEW_VARIANT,428,0.03556
ALK,0,MSK-IMPACT,hg19,2,29543736,29543736,+,nonsynonymous_SNV,SNP,A,A,G,novel,P-0000004-T01-IM3,NA,A,A,c.1427T>C,p.V476A,193,0.06736,13,793,0.00000,0,78,102,3,10,0,UNLIKELY,NEW_VARIANT,428,0.03556
PDCD1,0,MSK-IMPACT,hg19,2,242793433,242793433,+,nonsynonymous_SNV,SNP,G,G,A,novel,P-0000004-T01-IM3,NA,G,G,c.644C>T,p.A215V,240,0.06250,15,712,0.00140,1,89,136,5,10,0,UNLIKELY,NEW_VARIANT,428,0.03556
MAP3K1,0,MSK-IMPACT,hg19,5,56177843,56177843,+,nonsynonymous_SNV,SNP,C,C,G,novel,P-0000004-T01-IM3,NA,C,C,c.2816C>G,p.S939C,382,0.03141,12,666,0.00000,0,200,170,8,4,0,UNLIKELY,NEW_VARIANT,428,0.03556
FLT4,0,MSK-IMPACT,hg19,5,180030313,180030313,+,nonsynonymous_SNV,SNP,C,C,A,novel,P-0000004-T01-IM3,NA,C,C,c.3971G>T,p.R1324L,385,0.06494,25,1112,0.00270,3,182,178,12,13,0,UNLIKELY,NEW_VARIANT,428,0.03556
FLT4,0,MSK-IMPACT,hg19,5,180051003,180051003,+,nonsynonymous_SNV,SNP,T,T,C,novel,P-0000004-T01-IM3,NA,T,T,c.1480A>G,p.T494A,295,0.07458,22,977,0.00102,1,116,157,8,14,0,UNLIKELY,NEW_VARIANT,428,0.03556


Time difference of 10.46762 secs

Get impact_annotated (impact annotated with click_annotvcf)...


Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,cDNA_change,HGVSp_Short,t_depth,t_vaf,t_alt_count,n_depth,n_vaf,n_alt_count,t_ref_plus_count,t_ref_neg_count,t_alt_plus_count,t_alt_neg_count,occurence_in_normals,confidence_class,variant_status,sample_coverage,minor_contamination
SPEN,0,MSK-IMPACT,hg19,1,16265908,16265908,+,nonsynonymous_SNV,SNP,A,A,T,novel,P-0000004-T01-IM3,NA,A,A,c.10981A>T,p.I3661F,473,0.15433,73,1071,0.00000,0,166,234,37,36,0,AUTO_OK,NEW_VARIANT,428,0.03556
ALK,0,MSK-IMPACT,hg19,2,29543736,29543736,+,nonsynonymous_SNV,SNP,A,A,G,novel,P-0000004-T01-IM3,NA,A,A,c.1427T>C,p.V476A,193,0.06736,13,793,0.00000,0,78,102,3,10,0,UNLIKELY,NEW_VARIANT,428,0.03556
PDCD1,0,MSK-IMPACT,hg19,2,242793433,242793433,+,nonsynonymous_SNV,SNP,G,G,A,novel,P-0000004-T01-IM3,NA,G,G,c.644C>T,p.A215V,240,0.06250,15,712,0.00140,1,89,136,5,10,0,UNLIKELY,NEW_VARIANT,428,0.03556
MAP3K1,0,MSK-IMPACT,hg19,5,56177843,56177843,+,nonsynonymous_SNV,SNP,C,C,G,novel,P-0000004-T01-IM3,NA,C,C,c.2816C>G,p.S939C,382,0.03141,12,666,0.00000,0,200,170,8,4,0,UNLIKELY,NEW_VARIANT,428,0.03556
FLT4,0,MSK-IMPACT,hg19,5,180030313,180030313,+,nonsynonymous_SNV,SNP,C,C,A,novel,P-0000004-T01-IM3,NA,C,C,c.3971G>T,p.R1324L,385,0.06494,25,1112,0.00270,3,182,178,12,13,0,UNLIKELY,NEW_VARIANT,428,0.03556
FLT4,0,MSK-IMPACT,hg19,5,180051003,180051003,+,nonsynonymous_SNV,SNP,T,T,C,novel,P-0000004-T01-IM3,NA,T,T,c.1480A>G,p.T494A,295,0.07458,22,977,0.00102,1,116,157,8,14,0,UNLIKELY,NEW_VARIANT,428,0.03556


Time difference of 1.335729 mins

Join impact and impact_annotated...


...

Time difference of 21.30742 secs

Filter impact...


,Hugo_Symbol,Chromosome,Start_Position,End_Position,Consequence,Variant_Type,Reference_Allele,Tumor_Seq_Allele2,Tumor_Sample_Barcode,cDNA_change,HGVSp_Short,t_depth,t_vaf,t_alt_count,n_depth,n_vaf,n_alt_count,t_ref_plus_count,t_ref_neg_count,t_alt_plus_count,t_alt_neg_count,occurence_in_normals,confidence_class,sample_coverage,mut_key,VAG_VT,VAG_GENE,VAG_cDNA_CHANGE,VAG_PROTEIN_CHANGE,VAG_EFFECT,VEP_Consequence,VEP_SYMBOL,VEP_HGVSc,VEP_HGVSp,VEP_Amino_acids,VEP_VARIANT_CLASS,VEP_BIOTYPE,VEP_IMPACT,VEP_CLIN_SIG,VEP_AF,VEP_MAX_AF,VEP_MAX_AF_POPS,VEP_gnomAD_AF,VEP_SIFT,VEP_PolyPhen,VEP_COSMIC_CNT,sample_mut_key,patient_key
34,TP53,17,7577515,7577515,nonsynonymous_SNV,SNP,T,G,P-0000012-T02-IM3,c.766A>C,p.T256P,227,0.50220,114,569,0.00000,0,59,54,58,56,0,AUTO_OK,344,17_7577515_T_G,Sub,TP53,c.766A>C,p.T256P,non_synonymous_codon,missense_variant,TP53,ENST00000269305.4:c.766A>C,ENSP00000269305.4:p.Thr256Pro,T/P,SNV,protein_coding,MODERATE,NA,NA,NA,NA,NA,deleterious(0),probably_damaging(0.999),1,P-0000012-T02-IM3_17_7577515_T_G,P-0000012
37,PIK3R3,1,46521514,46521514,nonsynonymous_SNV,SNP,G,C,P-0000012-T03-IM3,c.894C>G,p.I298M,733,0.17599,129,1243,0.00000,0,288,316,61,68,0,AUTO_OK,428,1_46521514_G_C,Sub,PIK3R3,c.1032C>G,p.I344M,non_synonymous_codon,missense_variant,PIK3R3,ENST00000262741.5:c.894C>G,ENSP00000262741.5:p.Ile298Met,I/M,SNV,protein_coding,MODERATE,NA,NA,NA,NA,NA,deleterious(0),benign(0.277),NA,P-0000012-T03-IM3_1_46521514_G_C,P-0000012
38,ATR,3,142178126,142178126,nonsynonymous_SNV,SNP,C,A,P-0000012-T03-IM3,c.7292G>T,p.R2431M,482,0.17427,84,581,0.00172,1,221,177,46,38,0,AUTO_OK,428,3_142178126_C_A,Sub,ATR,c.7292G>T,p.R2431M,non_synonymous_codon,missense_variant,ATR,ENST00000350721.4:c.7292G>T,ENSP00000343741.4:p.Arg2431Met,R/M,SNV,protein_coding,MODERATE,NA,NA,8.96e-06,gnomAD_NFE,4.063e-06,deleterious(0),probably_damaging(0.997),NA,P-0000012-T03-IM3_3_142178126_C_A,P-0000012
39,PDGFRA,4,55139732,55139732,nonsynonymous_SNV,SNP,T,A,P-0000012-T03-IM3,c.1393T>A,p.L465M,570,0.20351,116,811,0.00000,0,252,202,66,50,0,AUTO_OK,428,4_55139732_T_A,Sub,PDGFRA,c.1393T>A,p.L465M,non_synonymous_codon,missense_variant,PDGFRA,ENST00000257290.5:c.1393T>A,ENSP00000257290.5:p.Leu465Met,L/M,SNV,protein_coding,MODERATE,NA,NA,NA,NA,NA,deleterious(0.01),probably_damaging(0.965),NA,P-0000012-T03-IM3_4_55139732_T_A,P-0000012
40,FBXW7,4,153249542,153249542,splicing,SNP,C,A,P-0000012-T03-IM3,c.1237-1G>T,,333,0.25526,85,458,0.00000,0,69,179,24,61,0,AUTO_OK,428,4_153249542_C_A,Sub,FBXW7,c.1237-1G>T,p.?,splice_site_variant,splice_acceptor_variant,FBXW7,ENST00000281708.4:c.1237-1G>T,NA,NA,SNV,protein_coding,HIGH,NA,NA,NA,NA,NA,NA,NA,NA,P-0000012-T03-IM3_4_153249542_C_A,P-0000012
41,FBXW7,4,153332775,153332775,stopgain_SNV,SNP,C,A,P-0000012-T03-IM3,c.181G>T,p.G61*,570,0.22807,130,908,0.00000,0,243,197,70,60,0,AUTO_OK,428,4_153332775_C_A,Sub,FBXW7,c.181G>T,p.G61*,stop_gained,stop_gained,FBXW7,ENST00000281708.4:c.181G>T,ENSP00000281708.3:p.Gly61Ter,G/*,SNV,protein_coding,HIGH,NA,NA,NA,NA,NA,NA,NA,NA,P-0000012-T03-IM3_4_153332775_C_A,P-0000012


Time difference of 51.94317 secs

Process raw features...


,Hugo_Symbol,Chromosome,Start_Position,End_Position,Consequence,Variant_Type,Reference_Allele,Tumor_Seq_Allele2,Tumor_Sample_Barcode,cDNA_change,HGVSp_Short,t_depth,t_vaf,t_alt_count,n_depth,n_vaf,n_alt_count,t_ref_plus_count,t_ref_neg_count,t_alt_plus_count,t_alt_neg_count,confidence_class,sample_coverage,mut_key,VAG_VT,VAG_GENE,VAG_cDNA_CHANGE,VAG_PROTEIN_CHANGE,VAG_EFFECT,VEP_Consequence,VEP_SYMBOL,VEP_HGVSc,VEP_HGVSp,VEP_Amino_acids,VEP_VARIANT_CLASS,VEP_BIOTYPE,VEP_IMPACT,VEP_CLIN_SIG,VEP_AF,VEP_MAX_AF,VEP_MAX_AF_POPS,VEP_gnomAD_AF,VEP_COSMIC_CNT,sample_mut_key,patient_key,frequency_in_normals,VEP_SIFT_class,VEP_SIFT_score,VEP_PolyPhen_class,VEP_PolyPhen_score,VEP_COSMIC
34,TP53,17,7577515,7577515,nonsynonymous_SNV,SNP,T,G,P-0000012-T02-IM3,c.766A>C,p.T256P,227,0.50220,114,569,0.00000,0,59,54,58,56,AUTO_OK,344,17_7577515_T_G,Sub,TP53,c.766A>C,p.T256P,non_synonymous_codon,missense_variant,TP53,c.766A>C,p.T256P,T/P,SNV,protein_coding,MODERATE,unknown,0,0.00e+00,unknown,0.000e+00,1,P-0000012-T02-IM3_17_7577515_T_G,P-0000012,0,deleterious,0.00,probably_damaging,0.999,1
37,PIK3R3,1,46521514,46521514,nonsynonymous_SNV,SNP,G,C,P-0000012-T03-IM3,c.894C>G,p.I298M,733,0.17599,129,1243,0.00000,0,288,316,61,68,AUTO_OK,428,1_46521514_G_C,Sub,PIK3R3,c.1032C>G,p.I344M,non_synonymous_codon,missense_variant,PIK3R3,c.894C>G,p.I298M,I/M,SNV,protein_coding,MODERATE,unknown,0,0.00e+00,unknown,0.000e+00,unknown,P-0000012-T03-IM3_1_46521514_G_C,P-0000012,0,deleterious,0.00,benign,0.277,0
38,ATR,3,142178126,142178126,nonsynonymous_SNV,SNP,C,A,P-0000012-T03-IM3,c.7292G>T,p.R2431M,482,0.17427,84,581,0.00172,1,221,177,46,38,AUTO_OK,428,3_142178126_C_A,Sub,ATR,c.7292G>T,p.R2431M,non_synonymous_codon,missense_variant,ATR,c.7292G>T,p.R2431M,R/M,SNV,protein_coding,MODERATE,unknown,0,8.96e-06,gnomAD_NFE,4.063e-06,unknown,P-0000012-T03-IM3_3_142178126_C_A,P-0000012,0,deleterious,0.00,probably_damaging,0.997,0
39,PDGFRA,4,55139732,55139732,nonsynonymous_SNV,SNP,T,A,P-0000012-T03-IM3,c.1393T>A,p.L465M,570,0.20351,116,811,0.00000,0,252,202,66,50,AUTO_OK,428,4_55139732_T_A,Sub,PDGFRA,c.1393T>A,p.L465M,non_synonymous_codon,missense_variant,PDGFRA,c.1393T>A,p.L465M,L/M,SNV,protein_coding,MODERATE,unknown,0,0.00e+00,unknown,0.000e+00,unknown,P-0000012-T03-IM3_4_55139732_T_A,P-0000012,0,deleterious,0.01,probably_damaging,0.965,0
40,FBXW7,4,153249542,153249542,splicing,SNP,C,A,P-0000012-T03-IM3,c.1237-1G>T,,333,0.25526,85,458,0.00000,0,69,179,24,61,AUTO_OK,428,4_153249542_C_A,Sub,FBXW7,c.1237-1G>T,p.?,splice_site_variant,splice_acceptor_variant,FBXW7,c.1237-1G>T,unknown,unknown,SNV,protein_coding,HIGH,unknown,0,0.00e+00,unknown,0.000e+00,unknown,P-0000012-T03-IM3_4_153249542_C_A,P-0000012,0,unknown,NA,unknown,NA,0
41,FBXW7,4,153332775,153332775,stopgain_SNV,SNP,C,A,P-0000012-T03-IM3,c.181G>T,p.G61*,570,0.22807,130,908,0.00000,0,243,197,70,60,AUTO_OK,428,4_153332775_C_A,Sub,FBXW7,c.181G>T,p.G61*,stop_gained,stop_gained,FBXW7,c.181G>T,p.G61*,G/*,SNV,protein_coding,HIGH,unknown,0,0.00e+00,unknown,0.000e+00,unknown,P-0000012-T03-IM3_4_153332775_C_A,P-0000012,0,unknown,NA,unknown,NA,0


Time difference of 1.109568 mins

 194210